# Representaiton of the basis and basic bitwise operators

## Variable types and binary representation

Default is 64-bit representation

In [1]:
typeof(1), typeof(1.)

(Int64, Float64)

But we can explicitly construct other types

In [2]:
x = Int8(1)
bitstring(x)

"00000001"

***Use signed integers with caution!***

In [3]:
bitstring(Int8(127)), bitstring(Int8(-128)), bitstring(Int8(-1))

("01111111", "10000000", "11111111")

In [4]:
z = parse(UInt8, "11111111", base=2)
print(z)

255

In [5]:
x = UInt8(42)
bitstring(x), Base.count_ones(x)

("00101010", 3)

## Bitwise operations

In [6]:
x = UInt8(1)
y = x << 2
print(y)
bitstring(x), bitstring(y)

4

("00000001", "00000100")

In [7]:
bitstring(x & y), bitstring(x | y)

("00000000", "00000101")

XOR (exclusive or) (to type use \xor + tab) -> useful for spin-flip operations

In [8]:
z = x | y
f = parse(UInt8, "111", base=2)
bitstring(z), bitstring(f), bitstring(z ⊻ f)

("00000101", "00000111", "00000010")

# S=1/2 Heisenberg chain

We'll now consider the 1D Heisenberg Hamiltonian $H = J \sum_{i=1}^{N-1} \vec{S}_i \cdot \vec{S}_{i+1}$

Denoting $S^\alpha = \frac{1}{2}\sigma^\alpha$, where $\sigma^\alpha$ ($\alpha=x,y,z$) are the Pauli matrices we have $H_i = J \sum_i \left[ \frac{1}{2}\left(\sigma^+_i \sigma^-_{i+1} +\sigma^-_i \sigma^+_{i+1}\right) + \frac{1}{4} \sigma^z_i \sigma^z_{i+1} \right]$

## Explicit construction of the Hamiltonian

In [9]:
using SparseArrays

In [10]:
"Flip spins at positions i and j in state x"
function FlipSpins(x,i,j)
    f = typeof(x)(1)<<(i-1) | typeof(x)(1)<<(j-1) 
    return x ⊻ f
end

FlipSpins

In [11]:
x = UInt8(2)
xf = FlipSpins(x, 1, 2)
bitstring(x), bitstring(xf)

("00000010", "00000001")

In [12]:
function ConstructHeisenbergHamiltonian(N)
    
    H = spzeros(2^N,2^N) # sparse matrix, will actually store only non-zero values, 
                         # i.e. no storage is allocated upon construction

    # Iterate over all the states in the basis |a> and calculate the nonzero matrix elements <b|H|a>
    # Recall that H = \sum_i H_{i,i+1}
    for a in range(0, length = 2^N)

        for i in range(1, length = N)
            j = mod(i,N)+1 # periodic boundary conditions
            si = ( a & 1<<(i-1) ) >> (i-1)
            sj = ( a & 1<<(j-1) ) >> (j-1)

            if si == sj
                H[a+1,a+1] += 1/4 # indexing in Julia is 1-based !!!
            else
                H[a+1,a+1] += - 1/4
                b = FlipSpins(a,i,j)
                H[a+1,b+1] = 1/2
            end
        end
    end
    
    return H
end

ConstructHeisenbergHamiltonian (generic function with 1 method)

In [13]:
N = 12
@time H = ConstructHeisenbergHamiltonian(N);

  0.248969 seconds (360.84 k allocations: 20.570 MiB, 6.17% gc time)


In [14]:
using LinearAlgebra

In [15]:
fullH = Matrix(H);

In [23]:
@time En = eigvals(fullH);
println("Ground state energy is ", En[1])

 12.462567 seconds (14 allocations: 129.469 MiB, 0.41% gc time)
Ground state energy is -5.387390917445213


Full diagonalization scales as $t\sim O({\rm dim} H ^3$)

In [20]:
using Arpack

In [22]:
@time res = eigs(H; nev=10, which=:LM, ritzvec=false) # Using Lanczos to calculate lowes!
En = res[1]
println("Ground state energy is ", En[1])

  0.159019 seconds (1.80 k allocations: 912.234 KiB)
Ground state energy is -5.38739091744521


Diagonalization with Lanczos scales as $t\sim O({\rm dim} H)$, i.e. factor of 2 for each extra spin